In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IcebergTimeTravel") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", "/tmp/iceberg_warehouse") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2") \
    .getOrCreate()

In [2]:
# Add a new column to the table
spark.sql("ALTER TABLE local.db.sample_users ADD COLUMNS (email STRING)")

DataFrame[]

In [3]:
from datetime import date

# Insert new data that includes the new column
new_data = [(3, "Nina", date(2024, 3, 1), "nina@example.com")]
df_new = spark.createDataFrame(new_data, ["id", "name", "signup_date", "email"])

df_new.writeTo("local.db.sample_users").append()

In [4]:
# Read full table — older rows will have nulls for 'email'
df = spark.read.table("local.db.sample_users")
df.show(truncate=False)

+---+--------+-----------+----------------+
|id |name    |signup_date|email           |
+---+--------+-----------+----------------+
|4  |Neo     |2024-04-01 |NULL            |
|3  |Nina    |2024-03-01 |nina@example.com|
|3  |Morpheus|2024-03-01 |NULL            |
|2  |Alex    |2024-02-01 |NULL            |
|1  |Trinity |2024-01-01 |NULL            |
|5  |Smith   |2024-05-01 |NULL            |
|6  |Anderson|2024-06-01 |NULL            |
+---+--------+-----------+----------------+

